# Automated Text Generation

In [1]:
import tensorflow as tf

In [2]:
# display tf version and test if GPU is active

tf.__version__, tf.test.gpu_device_name()

('2.6.0', '/device:GPU:0')

# Generating Text with a RNN

# Mount Google Drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Read the Corpus into Memory

In [4]:
# Read corpus into memory 

two_cities = "drive/My Drive/Colab Notebooks/two_cities.txt"

with open(two_cities) as f:
  corpus = f.read()

# Verify Corpus

In [5]:
# display text from the beginning of the corpus

print(corpus[:74])

A TALE OF TWO CITIES

A STORY OF THE FRENCH REVOLUTION

By Charles Dickens


In [6]:
# Get the length of the corpus to verify the end

len(corpus)

757247

In [7]:
print(corpus[757116:])

“It is a far, far better thing that I do, than I have ever done; it is a
far, far better rest that I go to than I have ever known.”


# Create Vocabulary

In [8]:
# Create a vocabulary of unique characters containded in the corpus

#unique characters in the corpus

vocab = sorted(set(corpus))
print("{} unique characters".format(len(vocab)))

74 unique characters


# Vectorize the text

# Create Integer Mappings

In [9]:
# Create the dictionary

# create a dictionary with integer representations of characters

int_map = {key: value for value, key in enumerate(vocab)}

int_map["a"]

45

# Create Character Mappings

In [10]:
#create the numpy array that holds character mappings

# create numpy array to hold character mappings of integers

import numpy as np


char_map = np.array(vocab)
char_map[45]

'a'

# Map a Sequence

In [11]:
 # create a variable to hold line break

 br = "\n"
 

In [12]:
#simple sequence

sequence = "hello world"
print("original sequence:",sequence,br)

original sequence: hello world 



In [13]:
# map to integer representations

maps = np.array([int_map[c] for c in sequence])
print("integer mappings:",maps,br)

integer mappings: [52 49 56 56 59  1 67 59 62 56 48] 



In [14]:
# map integer representations back into characters

s = [char_map[i] for i in maps]

In [15]:
# create string from list of characters

s = "".join(s)
print("translations:",s)

translations: hello world


# Vectorize the Corpus

In [16]:
#vectorize the corpus

encoded = np.array([int_map[c] for c in corpus])
encoded[:20],char_map[encoded[:20]]

(array([19,  1, 38, 19, 30, 23,  1, 33, 24,  1, 38, 41, 33,  1, 21, 27, 38,
        27, 23, 37]),
 array(['A', ' ', 'T', 'A', 'L', 'E', ' ', 'O', 'F', ' ', 'T', 'W', 'O',
        ' ', 'C', 'I', 'T', 'I', 'E', 'S'], dtype='<U1'))

# Create Training input Sequences

In [17]:
# Coverting the encoded corpus into tensorflow tensors

#initialize maximum length sequence for a single input

seq_length = 100

In [18]:
# create training dataset
ds = tf.data.Dataset.from_tensor_slices(encoded)
ds

<TensorSliceDataset shapes: (), types: tf.int64>

# display Samples

In [19]:
#display samples from the training dataset

for i in ds.take(6):
  print(i.numpy(),":",char_map[i])

19 : A
1 :  
38 : T
19 : A
30 : L
23 : E


# Batch Sequences

In [20]:
sequences = ds.batch(seq_length+1, drop_remainder=True)

In [21]:
#display the first batch

for i in sequences.take(1):
  print(char_map[i],br)
  print("batch size:",len(i))

['A' ' ' 'T' 'A' 'L' 'E' ' ' 'O' 'F' ' ' 'T' 'W' 'O' ' ' 'C' 'I' 'T' 'I'
 'E' 'S' '\n' '\n' 'A' ' ' 'S' 'T' 'O' 'R' 'Y' ' ' 'O' 'F' ' ' 'T' 'H' 'E'
 ' ' 'F' 'R' 'E' 'N' 'C' 'H' ' ' 'R' 'E' 'V' 'O' 'L' 'U' 'T' 'I' 'O' 'N'
 '\n' '\n' 'B' 'y' ' ' 'C' 'h' 'a' 'r' 'l' 'e' 's' ' ' 'D' 'i' 'c' 'k' 'e'
 'n' 's' '\n' '\n' '\n' 'C' 'O' 'N' 'T' 'E' 'N' 'T' 'S' '\n' '\n' '\n' ' '
 ' ' ' ' ' ' ' ' 'B' 'o' 'o' 'k' ' ' 't' 'h' 'e'] 

batch size: 101


# Create Sample and targets

In [22]:
#build a function that creates sample target sets

def create_sample_target(piece):
  sample = piece[:-1]
  target = piece[1:]
  return sample, target

In [23]:
# Create the dataset

dataset = sequences.map(create_sample_target)

In [24]:
#Displaying the sample and target from the first input sequence

for sample, target in dataset.take(1):
  print("sample:",char_map[sample],br)
  print("target:",char_map[target])

sample: ['A' ' ' 'T' 'A' 'L' 'E' ' ' 'O' 'F' ' ' 'T' 'W' 'O' ' ' 'C' 'I' 'T' 'I'
 'E' 'S' '\n' '\n' 'A' ' ' 'S' 'T' 'O' 'R' 'Y' ' ' 'O' 'F' ' ' 'T' 'H' 'E'
 ' ' 'F' 'R' 'E' 'N' 'C' 'H' ' ' 'R' 'E' 'V' 'O' 'L' 'U' 'T' 'I' 'O' 'N'
 '\n' '\n' 'B' 'y' ' ' 'C' 'h' 'a' 'r' 'l' 'e' 's' ' ' 'D' 'i' 'c' 'k' 'e'
 'n' 's' '\n' '\n' '\n' 'C' 'O' 'N' 'T' 'E' 'N' 'T' 'S' '\n' '\n' '\n' ' '
 ' ' ' ' ' ' ' ' 'B' 'o' 'o' 'k' ' ' 't' 'h'] 

target: [' ' 'T' 'A' 'L' 'E' ' ' 'O' 'F' ' ' 'T' 'W' 'O' ' ' 'C' 'I' 'T' 'I' 'E'
 'S' '\n' '\n' 'A' ' ' 'S' 'T' 'O' 'R' 'Y' ' ' 'O' 'F' ' ' 'T' 'H' 'E' ' '
 'F' 'R' 'E' 'N' 'C' 'H' ' ' 'R' 'E' 'V' 'O' 'L' 'U' 'T' 'I' 'O' 'N' '\n'
 '\n' 'B' 'y' ' ' 'C' 'h' 'a' 'r' 'l' 'e' 's' ' ' 'D' 'i' 'c' 'k' 'e' 'n'
 's' '\n' '\n' '\n' 'C' 'O' 'N' 'T' 'E' 'N' 'T' 'S' '\n' '\n' '\n' ' ' ' '
 ' ' ' ' ' ' 'B' 'o' 'o' 'k' ' ' 't' 'h' 'e']


# Time Step Prediction

In [25]:
#showing the first 5 time steps from the sample and target sets

for i, (input_idx,target_idx) in enumerate(zip(sample[:5],target[:5])):
  print("step:",i)
  print("input:",input_idx.numpy(),char_map[input_idx])
  print("expected out:",target_idx.numpy(),char_map[target_idx])
  if i < 4:
    print()

step: 0
input: 19 A
expected out: 1  

step: 1
input: 1  
expected out: 38 T

step: 2
input: 38 T
expected out: 19 A

step: 3
input: 19 A
expected out: 30 L

step: 4
input: 30 L
expected out: 23 E


# Create Training Batches

In [26]:
#Set batch and buffer sizes

BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [27]:
#Shuffle, batch,cache and prefetch

corpus_ds = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True).cache().prefetch(1)
corpus_ds

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build the Model

In [28]:
# Length of the vocabulary in chars

vocab_size = len(vocab)

In [29]:
#the embedding dimension
embedding_dim = 256

In [30]:
#number of RNN units
rnn_units = 1024

In [31]:
# Create the model

# generate seed for reproducibility

tf.random.set_seed(0)
np.random.seed(0)

In [32]:
#clear any previous models

tf.keras.backend.clear_session()

In [33]:
#import libs

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU,Dense, Embedding
from tensorflow.keras import losses

In [34]:
#create the model

model = Sequential([Embedding(vocab_size,embedding_dim,batch_input_shape=[BATCH_SIZE,None]),GRU(rnn_units,return_sequences=True,stateful=True,recurrent_initializer="glorot_uniform"),Dense(vocab_size)])

# Display Model Summary

In [35]:
#inspect the model

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           18944     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 74)            75850     
Total params: 4,033,098
Trainable params: 4,033,098
Non-trainable params: 0
_________________________________________________________________


# Checking the Output Shape

In [36]:
#display the shape of the first batch in the dataset

for sample, target in corpus_ds.take(1):
  example_batch_predictions = model(sample)

example_batch_predictions.shape

TensorShape([64, 100, 74])

# Calculate Loss

In [37]:
# Building a function to calculate loss

def loss(labels,logits):
  return losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [38]:
# Invoking the function

pre_trained_loss = loss(target,example_batch_predictions)

In [39]:
#checking the shape

print("pred shape:",example_batch_predictions.shape)
print("scalar_loss",pre_trained_loss.numpy().mean())

pred shape: (64, 100, 74)
scalar_loss 4.3027983


# compile the Model

In [40]:
# compile the model

model.compile(loss=loss,optimizer="adam")

# Configure Checkpoints

In [41]:
# Saving checkpoints so we can recall what the RNN has learned

import os


In [42]:
# directory where the checkpoints will be saved

checkpoint_dir = "./training_checkpoints"

In [43]:
#name of the checkpoint files

checkpoint_files = os.path.join(checkpoint_dir,"ckpt_{epoch}")

In [44]:
#callback method

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_files,save_weights_only=True)

# Train the Model

In [45]:
# Train the model for ten epochs

EPOCHS = 10

history = model.fit(corpus_ds,epochs=EPOCHS,callbacks=[checkpoint_callback])

Epoch 1/10
117/117 [==============================] - 7s 39ms/step - loss: 2.8478
Epoch 2/10
117/117 [==============================] - 5s 39ms/step - loss: 2.1239
Epoch 3/10
117/117 [==============================] - 5s 39ms/step - loss: 1.8559
Epoch 4/10
117/117 [==============================] - 5s 39ms/step - loss: 1.6671
Epoch 5/10
117/117 [==============================] - 5s 39ms/step - loss: 1.5358
Epoch 6/10
117/117 [==============================] - 5s 39ms/step - loss: 1.4416
Epoch 7/10
117/117 [==============================] - 5s 39ms/step - loss: 1.3696
Epoch 8/10
117/117 [==============================] - 5s 39ms/step - loss: 1.3117
Epoch 9/10
117/117 [==============================] - 5s 39ms/step - loss: 1.2615
Epoch 10/10
117/117 [==============================] - 5s 39ms/step - loss: 1.2143


# Rebuild the model for Text Creation

## Restore Weights from Checkpoints

In [46]:
#Restore

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

## Rebuild with Batch Size of 1

In [47]:
# Rebuild the model with batch size of 1 

#generate seed for reproducibility

tf.random.set_seed(0)
np.random.seed(0)

In [48]:
# clear any previous models

tf.keras.backend.clear_session()

In [49]:
# Set batch size to 1

BATCH_SIZE = 1



In [50]:
#Rebuild model

model = Sequential([
  Embedding(vocab_size, embedding_dim,
            batch_input_shape=[BATCH_SIZE, None]),
  GRU(rnn_units, return_sequences=True,
      stateful=True, recurrent_initializer='glorot_uniform'),
  Dense(vocab_size)
])

In [51]:
# Load Weights and Reshape


model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

# Model Summary

In [52]:
#inspect the model

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            18944     
_________________________________________________________________
gru (GRU)                    (1, None, 1024)           3938304   
_________________________________________________________________
dense (Dense)                (1, None, 74)             75850     
Total params: 4,033,098
Trainable params: 4,033,098
Non-trainable params: 0
_________________________________________________________________


# Create Components to Generate Text

In [53]:
# Create the text generation function

def create_text(model, input_eval, temperature, start_string):

  # Empty string to store our results
  new_text = []

  # Here batch size == 1
  model.reset_states()

  for i in range(n):
    # model encoded input
    predictions = model(input_eval)

    # remove batch dimension so we can manipulate predictions
    predictions = tf.squeeze(predictions, 0)

    # divide predictions by temperature
    predictions = predictions / temperature

    # use a categorical distribution to predict character
    # returned by model
    predicted_id = tf.random.categorical(
        predictions, num_samples=1)[-1,0].numpy()

    # pass predicted character as next input to model
    # with previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    # append generated characters to text
    new_text.append(char_map[predicted_id])

  return (start_string + ''.join(new_text))

# Initialize Variables

In [54]:
# initialize Variables

n = 500
temp = 0.3
start_string = "Tale"


# Vectorize and Reshape the starting String

In [55]:
# Vectorize starting string

input_vectorized = [int_map[s] for s in start_string]
print("original shape:",end=" ")
print(str(np.array(input_vectorized).ndim) + "D",br)

original shape: 1D 



In [56]:
# reshape string for TensorFlow model consumption


input_vectorized = tf.expand_dims(input_vectorized, 0 )
print("new shape:",input_vectorized.shape)

new shape: (1, 4)


# Create New Text

In [57]:
#generate random seeds

tf.random.set_seed(0)
np.random.seed(0)

In [59]:
print(create_text(model,input_vectorized,temp,start_string))

Tale, Mr. Lorry, he made a long bank, and they were in the old man, with a sudden to the postilions of the face, and with the countenance and
the sun at the truth.”

“I think it is my father, I shall say that I have seen the truth, with the first time. There was a stare and the fact, with his hand, and the confidence with an end of the window, with the shoemaker's
eyes were so far as the face of my father, she had not been a strong and mother of the courtyard, and the honour of the people was a poor
